In [3]:
import pandas as pd
import os

In [10]:
dfs = []
for file in os.listdir('.'):
    if file.endswith('.xlsx') and not file.startswith('_'):
        source_df = pd.read_excel(file)
        dfs.append(source_df)
df = pd.concat(dfs, ignore_index=True)
df = df.fillna('')

In [11]:
len(df)

3108

In [5]:
third_words = set(df[df['Nombre aceptado'].str.count(' ') > 2]['Nombre aceptado'].apply(lambda x: x.split(' ')[2] if
                                                                                        x.split(' ')[2][-1] == '.' and
                                                                                        x.split(' ')[2][0] != '(' and
                                                                                        x.split(' ')[2] == x.split(' ')[2].lower()
                                                                                        else '').tolist())
third_words

{'', 'f.', 'subsp.', 'var.'}

In [ ]:
df_taxonomia = pd.read_excel('../Datos EIDOS/EIDOS_taxonomia.xlsx')

In [41]:
# Quitar parentesis y palabras que empiezan por mayuscula y numeros
df_taxonomia['lowered_name'] = df_taxonomia['name'].str.replace(r'[()&]', '', regex=True).apply(
    lambda x: x[0].lower() + ' '.join([w for w in x[1:].split() if w == w.lower() and not w.isdigit()])
    )

def find_best_match(row):
    if row['Código de la especie (id taxon)'] != '':
        return row['Código de la especie (id taxon)'] # Si tiene codigo de especie, no necesitamos buscar
    row_words = row['Nombre aceptado'].replace('(', '').replace(')', '').lower().split()
    if len(row_words) < 2:
        return None # No hay suficientes palabras para buscar
    # Filtrar candidatos
    candidates = df_taxonomia[df_taxonomia['lowered_name'].str.contains(' '.join(row_words[:2]))]
    # Si hay una categoría taxonómica, refinar los candidatos
    if len(row_words) > 3 and row_words[2] in ['f.', 'subsp.', 'var.']:
        candidates = candidates[candidates['lowered_name'].str.contains(f' {row_words[3]} ')]
    else:
        candidates = candidates[~candidates['lowered_name'].str.contains(r'\b(?: f\. | subsp\. | var\. )\b', regex=True)]
    if candidates.empty:
        return None
    # Calcular las puntuaciones de coincidencia
    row_words_set = set(row_words)
    candidates['score'] = candidates['lowered_name'].apply(lambda c: 
        0.9 * len(set(c.split()) & row_words_set) / len(row_words) +
        0.1 * len(set(c.split()) & row_words_set) / len(c.split())
    )
    # Obtener el mejor candidato
    best_match = candidates.loc[candidates['score'].idxmax()]
    return best_match['taxonid']

df['Código de la especie (id taxon)'] = df.apply(find_best_match, axis=1).tolist()
df.to_excel('_DESCRIPCIONES_TODAS.xlsx', index=False)